In [1]:
import pandas as pd

In [3]:
train_df = pd.read_csv('C:/SOOKTAT/0. 프로젝트/Taxi/data/train.csv')

In [4]:
train_df.shape

(55423856, 8)

* 결측치 제거

In [5]:
train_df.dropna(inplace=True)

train_df.shape

(55423480, 8)

* 위도, 경도 이상치 제거

In [6]:
train_df = train_df.drop((train_df[train_df['pickup_latitude']<=39]).index, axis=0)
train_df = train_df.drop((train_df[train_df['pickup_latitude']>=46]).index, axis=0)

In [7]:
train_df = train_df.drop((train_df[train_df['dropoff_latitude']<=39]).index, axis=0)
train_df = train_df.drop((train_df[train_df['dropoff_latitude']>=46]).index, axis=0)

In [8]:
train_df = train_df.drop((train_df[train_df['pickup_longitude']>=-71]).index, axis=0)
train_df = train_df.drop((train_df[train_df['pickup_longitude']<=-80]).index, axis=0)

In [9]:
train_df = train_df.drop((train_df[train_df['dropoff_longitude']>=-71]).index, axis=0)
train_df = train_df.drop((train_df[train_df['dropoff_longitude']<=-80]).index, axis=0)

In [10]:
train_df.shape

(54272275, 8)

* fare_amount = 0 제거

In [11]:
train_df = train_df.drop(train_df[train_df['fare_amount']<0].index, axis=0)

train_df.shape

(54270068, 8)

* passenger_count 이상치 제거

In [12]:
train_df = train_df.drop((train_df[train_df['passenger_count']==0]).index, axis=0)
train_df = train_df.drop((train_df[train_df['passenger_count']>6]).index, axis=0)

train_df.shape

(54079157, 8)

* 거리 계산

In [13]:
pickup=train_df[['pickup_latitude', 'pickup_longitude']]
pickup = pd.DataFrame(pickup)

pickup.head()

,pickup_latitude,pickup_longitude
0,40.721319,-73.844311
1,40.711303,-74.016048
2,40.761270,-73.982738
3,40.733143,-73.987130
4,40.768008,-73.968095


In [14]:
pickup.shape

(54079157, 2)

In [15]:
dropoff=train_df[['dropoff_latitude', 'dropoff_longitude']]
dropoff=pd.DataFrame(dropoff)

dropoff.head()

,dropoff_latitude,dropoff_longitude
0,40.712278,-73.841610
1,40.782004,-73.979268
2,40.750562,-73.991242
3,40.758092,-73.991567
4,40.783762,-73.956655


In [16]:
dropoff.shape

(54079157, 2)

In [17]:
import geopy.distance

In [19]:
from geopy import distance

In [20]:
dist = []

for i in range(len(dropoff.index)) : 
 result = distance.distance(pickup.values[i], dropoff.values[i]).m
 dist.append(result)

In [21]:
dist=pd.DataFrame(dist)

dist.head()

,0
0,1029.600744
1,8443.441363
2,1389.132160
3,2795.789658
4,1998.337836


In [22]:
dist.shape

(54079157, 1)

* dist와 fare_amount와의 관계

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(17,6))
plt.scatter(train_df['fare_amount'], dist)
plt.show()

In [ ]:
corr(train_df['fare_amount'], dist)

* pickup_datetime 쪼개기

In [23]:
train_df['key'] = pd.to_datetime(train_df['key'])
train_df['pickup_datetime']  = pd.to_datetime(train_df['pickup_datetime'])

In [24]:
for i in range(len(train_df)) :
    train_df['Year'] = train_df['pickup_datetime'].dt.year
    train_df['Month'] = train_df['pickup_datetime'].dt.month
    train_df['Date'] = train_df['pickup_datetime'].dt.day
    train_df['Day of Week'] = train_df['pickup_datetime'].dt.dayofweek
    train_df['Hour'] = train_df['pickup_datetime'].dt.hour

KeyboardInterrupt: 

In [ ]:
train_df = train_df.drop(['key', 'pickup_datetime'], axis = 1)

train_df.head()

* taxi_amount 생성

In [29]:
train_df['taxi_amount'] = [1 if i<=4 else 2 for i in train_df['passenger_count']]

train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,taxi_amount
0,2009-06-15 17:26:21.000000100,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,0,17,1
1,2010-01-05 16:52:16.000000200,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,1,16,1
2,2011-08-18 00:35:00.000000490,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,3,0,1
3,2012-04-21 04:30:42.000000100,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,5,4,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,1,7,1


In [31]:
train_df['taxi_amount'].value_counts()

1    49086686
2     4992471
Name: taxi_amount, dtype: int64

* fare_amount_ontaxi 생성

In [30]:
train_df['fare_amount_ontaxi'] = train_df['fare_amount'] / train_df['taxi_amount']

train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,taxi_amount,fare_amount_ontaxi
0,2009-06-15 17:26:21.000000100,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,0,17,1,4.5
1,2010-01-05 16:52:16.000000200,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,1,16,1,16.9
2,2011-08-18 00:35:00.000000490,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,3,0,1,5.7
3,2012-04-21 04:30:42.000000100,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,5,4,1,7.7
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,1,7,1,5.3


* 모델 공부

https://velog.io/@anjoy/ML-%EA%B8%B0%EC%B4%88-%EC%8C%93%EA%B8%B0-3-%EC%A7%80%EB%8F%84-%ED%95%99%EC%8A%B5

* 최종 데이터 저장

In [32]:
train_df.to_csv("C:/SOOKTAT/0. 프로젝트/Taxi/data/train_fin.csv")

In [33]:
dist.to_csv("C:/SOOKTAT/0. 프로젝트/Taxi/data/distance.csv")

In [37]:
train_df.isnull().sum()

key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
Year                  0
Month                 0
Date                  0
Day of Week           0
Hour                  0
taxi_amount           0
fare_amount_ontaxi    0
dtype: int64

In [40]:
ex = pd.concat([train_df[:10], dist[:10]], axis = 1)

ex

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Year,Month,Date,Day of Week,Hour,taxi_amount,fare_amount_ontaxi,0
0,2009-06-15 17:26:21.000000100,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,0,17,1,4.5,1029.600744
1,2010-01-05 16:52:16.000000200,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,1,16,1,16.9,8443.441363
2,2011-08-18 00:35:00.000000490,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,3,0,1,5.7,1389.132160
3,2012-04-21 04:30:42.000000100,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,5,4,1,7.7,2795.789658
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,1,7,1,5.3,1998.337836
5,2011-01-06 09:50:45.000000200,12.1,2011-01-06 09:50:45+00:00,-74.000964,40.731630,-73.972892,40.758233,1,2011,1,6,3,9,1,12.1,3787.980069
6,2012-11-20 20:35:00.000000100,7.5,2012-11-20 20:35:00+00:00,-73.980002,40.751662,-73.973802,40.764842,1,2012,11,20,1,20,1,7.5,1554.446280
7,2012-01-04 17:22:00.000000810,16.5,2012-01-04 17:22:00+00:00,-73.951300,40.774138,-73.990095,40.751048,1,2012,1,4,2,17,1,16.5,4159.930253
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00+00:00,-74.006462,40.726713,-73.993078,40.731628,1,2012,12,3,0,13,1,9.0,1255.507373
9,2009-09-02 01:11:00.000000830,8.9,2009-09-02 01:11:00+00:00,-73.980658,40.733873,-73.991540,40.758138,2,2009,9,2,2,1,1,8.9,2847.028411


In [42]:
dist.columns = ['distance']

dist

,distance
0,1029.600744
1,8443.441363
2,1389.132160
3,2795.789658
4,1998.337836
...,...
54079152,4339.870263
54079153,1070.156530
54079154,5069.449125
54079155,9369.566503


In [46]:
train_df2 = pd.concat([train_df, dist], axis = 1)

train_df2

MemoryError: Unable to allocate 423. MiB for an array with shape (55391294,) and data type int64

In [ ]:
train_df2.to_csv("C:/SOOKTAT/0. 프로젝트/Taxi/data/train_fin_dist.csv")